In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
main_domain = 'https://luxproject.luxoft.com'
main_urls = 'https://luxproject.luxoft.com/confluence/display/SSBI/Master+data+source+register'
#single_page_example = 'https://luxproject.luxoft.com/confluence/display/SSBI/SFDC+MDS+SNAPSHOTS'
login = '---' #поле для логина
password = '---' #поле для пороля

In [3]:
def get_bs_obj(url: str) :
    html_rsp = requests.get(f'{url}',auth=(login, password))
    return BeautifulSoup(html_rsp.text, 'lxml')

In [4]:
def collect_links(main_url):
    report_names = []
    report_links = []

    soup = get_bs_obj(main_url)
    table = soup.find('div',attrs={"class":"table-wrap conf-macro output-block"}).find('table').find('tbody')  

    rows = table.find_all('tr')
    for row in rows:
        report_name = row.find('a').text
        report_link = main_domain + row.find('a').get("href")
        report_names.append(report_name)
        report_links.append(report_link)
    return report_links

In [5]:
def collect_tags(single_page):
    tags_list = []
    soup_single_link = get_bs_obj(single_page)
    tags = soup_single_link.find('div',class_="labels-content")
    for tag in tags.find_all('li',class_='aui-label'):
          tag = tag.find('a').text
          tags_list.append(tag)
    return tags_list

In [6]:
def collect_single_page_data(single_link: str):
    header_report = ''
    object_name = ''
    status = ''
    importance = ''
    data_from_systems = ''
    DWH_data_steward = ''
    ra_data_steward = ''
    business_data_steward = ''
    update_schedule_SLA = ''
    description = ''
    ETL_package = ''
    shared_data_source = ''
    jira_epic_link = ''
    bitbucket_repository = ''
    last_verified_on = ''
    attrs_names = ['Object','Status','Importance','Data from systems','DWH Data Steward',\
                   'R&A Data Steward','Business Data Steward','Update Schedule SLA',\
                   'Description','ETL package','Shared Data Source','Jira Epic Link','Bitbucket repository','Last verified on']
    attrs_values = []
    soup = ''
    df = pd.DataFrame()

    #soup = get_bs_obj('https://luxproject.luxoft.com/confluence/pages/viewpage.action?pageId=476828105')
    soup = get_bs_obj(single_link)
    
    
    try:
        header_report = soup.find('h1',id="title-text").text.strip()
    except AttributeError:
        header_report = ''
        
    try:
        object_name = soup.find("th", text="Object name").find_next_sibling("td").text
    except AttributeError:
        object_name = ''

    try:
        status = soup.find("th", text="Status").find_next_sibling("td").text
    except AttributeError:
        status = ''

    try:
        importance = soup.find("th", text="Importance").find_next_sibling("td").text
    except AttributeError:
        importance = ''   

    try:
        data_from_systems = soup.find("th", text="Data from systems").find_next_sibling("td").text
    except AttributeError:
        data_from_systems = ''

    try:
        DWH_data_steward = soup.find("th", text="DWH Data Steward").find_next_sibling("td").text
    except AttributeError:
        DWH_data_steward = ''     

    try:
        ra_data_steward = soup.find("th", text="R&A Data Steward").find_next_sibling("td").text
    except AttributeError:
        ra_data_steward = ''

    try:
        business_data_steward = soup.find("th", text="Business Data Steward").find_next_sibling("td").text
    except AttributeError:
        business_data_steward = ''

    try:
        update_schedule_SLA = soup.find("th", text="Update Schedule SLA").find_next_sibling("td").text.replace('\xa0', '')
    except AttributeError:
        update_schedule_SLA = ''

    try:
        description = soup.find("th", text="Description").find_next_sibling("td").text
    except AttributeError:
        description = ''

    try:
        ETL_package = soup.find("th", text="ETL package").find_next_sibling("td").text
    except AttributeError:
        ETL_package = ''

    try:
        shared_data_source = soup.find("th", text="Shared data source").find_next_sibling("td").\
        find('a',attrs={'class':'external-link'}).get('href')
    except AttributeError:
        shared_data_source = ''

    try:
        jira_epic_link = soup.find("th", text="Jira Epic Link").find_next_sibling("td").\
        find('a',attrs={'class':'jira-issue-key'}).get('href')
    except AttributeError:
        jira_epic_link = ''   

    try:
        bitbucket_repository = soup.find("th", text="Bitbucket repository").find_next_sibling("td").text.strip()
    except AttributeError:
        bitbucket_repository = ''         

    try:
        last_verified_on = soup.find("th", text="Last verified on").find_next_sibling("td").text
    except AttributeError:
        last_verified_on = ''      

    attrs_values = [object_name,status,importance,data_from_systems,DWH_data_steward,\
                    ra_data_steward,business_data_steward,update_schedule_SLA,\
                    description,ETL_package,shared_data_source,jira_epic_link,bitbucket_repository,last_verified_on]


#     df = pd.DataFrame(list(zip(attrs_names, attrs_values)),
#                    columns =['attr_name', 'attr_value'])
#     df.insert(0, "jira_page_name", header_report)
#     df
    return header_report,attrs_names,attrs_values

In [7]:
df_to_sql = pd.DataFrame()
for i in range(len(collect_links(main_urls))): 
    single_df = pd.DataFrame()
    jira_link = collect_links(main_urls)[i]
    tags = collect_tags(collect_links(main_urls)[i])
    single_page_name = collect_single_page_data(collect_links(main_urls)[i])[0] 
    single_page_names = collect_single_page_data(collect_links(main_urls)[i])[1]
    single_page_values = collect_single_page_data(collect_links(main_urls)[i])[2]
    single_df = pd.DataFrame(list(zip(single_page_names, single_page_values)),
                   columns =['Attr_name', 'Attr_value'])
    single_df['Jira_link'], single_df['Tags'], single_df['Page_Name'] = [jira_link, ', '.join(tags), single_page_name]
    df_to_sql = pd.concat([df_to_sql, single_df], axis=0).reset_index(drop=True)
df_to_sql['Collect_Date'] = pd.to_datetime('today')
df_to_sql = df_to_sql[['Page_Name', 'Jira_link', 'Tags', 'Attr_name', 'Attr_value','Collect_Date']]
df_to_sql

,Page_Name,Jira_link,Tags,Attr_name,Attr_value,Collect_Date
0,SFDC MDS SNAPSHOTS,https://luxproject.luxoft.com/confluence/displ...,"mds-finsales, test1",Object,SFDC_MDS_SNAPSHOTS,2022-09-06 13:36:44.265719
1,SFDC MDS SNAPSHOTS,https://luxproject.luxoft.com/confluence/displ...,"mds-finsales, test1",Status,READY FOR USE,2022-09-06 13:36:44.265719
2,SFDC MDS SNAPSHOTS,https://luxproject.luxoft.com/confluence/displ...,"mds-finsales, test1",Importance,CRITICAL,2022-09-06 13:36:44.265719
3,SFDC MDS SNAPSHOTS,https://luxproject.luxoft.com/confluence/displ...,"mds-finsales, test1",Data from systems,Cognos; SaleForce; ED,2022-09-06 13:36:44.265719
4,SFDC MDS SNAPSHOTS,https://luxproject.luxoft.com/confluence/displ...,"mds-finsales, test1",DWH Data Steward,,2022-09-06 13:36:44.265719
...,...,...,...,...,...,...
93,UNIFIED FINANCIAL SOURCE PIVOT,https://luxproject.luxoft.com/confluence/displ...,mds-finsales,ETL package,"Package = COGNOS_UPLOAD_MAIN, Task = UNIFIED F...",2022-09-06 13:36:44.265719
94,UNIFIED FINANCIAL SOURCE PIVOT,https://luxproject.luxoft.com/confluence/displ...,mds-finsales,Shared Data Source,,2022-09-06 13:36:44.265719
95,UNIFIED FINANCIAL SOURCE PIVOT,https://luxproject.luxoft.com/confluence/displ...,mds-finsales,Jira Epic Link,https://luxproject.luxoft.com/jira/browse/TABL...,2022-09-06 13:36:44.265719
96,UNIFIED FINANCIAL SOURCE PIVOT,https://luxproject.luxoft.com/confluence/displ...,mds-finsales,Bitbucket repository,,2022-09-06 13:36:44.265719


In [8]:
from sqlalchemy import create_engine
import sqlalchemy 

server = 'oro-dwh-01'
database = 'DACTests'
engine = create_engine(f'mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [9]:
df_to_sql.to_sql('jira_tableau_MDSR'\
                 , con = engine\
                 , if_exists = 'replace'\
                 ,index=False)